# 0010_fastcore_meta_summary

## import

In [ ]:
from fastdebug.utils import *
from fastdebug.core import *

## fastcore and fastcore.meta

In [ ]:
import fastcore

In [ ]:
whichversion("fastcore")

fastcore: 1.5.27 
Python supercharged for fastai development    
Jeremy Howard and Sylvain Gugger 
https://github.com/fastai/fastcore/     
python_version: >=3.7     
/Users/Natsume/mambaforge/lib/python3.9/site-packages/fastcore


In [ ]:
whatinside(fastcore, lib=True)

The library has 21 modules
['_modidx',
 '_nbdev',
 'all',
 'basics',
 'dispatch',
 'docments',
 'docscrape',
 'foundation',
 'imports',
 'meta',
 'nb_imports',
 'net',
 'parallel',
 'script',
 'shutil',
 'style',
 'test',
 'transform',
 'utils',
 'xdg',
 'xtras']


In [ ]:
from fastcore.meta import *
import fastcore.meta as fm

### What's inside fastcore.meta

In [ ]:
whatinside(fm, dun=True)

fastcore.meta has: 
13 items in its __all__, and 
43 user defined functions, 
19 classes or class objects, 
2 builtin funcs and methods, and
74 callables.

test_sig:            function    Test the signature of an object
FixSigMeta:          metaclass, type    A metaclass that fixes the signature on classes that override `__new__`
PrePostInitMeta:     metaclass, type    A metaclass that calls optional `__pre_init__` and `__post_init__` methods
AutoInit:            class, PrePostInitMeta    Same as `object`, but no need for subclasses to call `super().__init__`
NewChkMeta:          metaclass, type    Metaclass to avoid recreating object passed to constructor
BypassNewMeta:       metaclass, type    Metaclass: casts `x` to this class if it's of type `cls._bypass_type`
empty2none:          function    Replace `Parameter.empty` with `None`
anno_dict:           function    `__annotation__ dictionary with `empty` cast to `None`, returning empty if doesn't exist
use_kwargs_dict:     decorator,

## Review individual funcs and classes

### What is fastcore.meta all about? 

It is a submodule contains 4 metaclasses, 1 class built by a metaclass, 4 decorators and a few functions.    

Metaclasses give us the power to create new breeds of classes with new features.     

Decorators give us the power to add new features to existing funcions.    

We can find their basic info [above](#What's-inside-fastcore.meta)

### What can these metaclasses do for me?

#### FixSigMeta
can breed classes which are free of signature problems (or they can automatically fix signature problems).

#### PrePostInitMeta
inherited/evolved from `FixSigMeta` to breed classes which can initialize their objects using `__pre_init__`, 
`__init__`, `__post_init__` whichever is available (allow me to abbreviate it as triple_init).

#### AutoInit
is an instance class created by `PrePostInitMeta`, and together with its own defined `__pre_init__`, subclasses of `AutoInit` has to worry about running `super().__init__(...)` no more.

- As `AutoInit` is an instance class created by `PrePostInitMeta`, it can pass on both features (free of signature problem and triple_init) to its subclasses. 
- As it also defines its own `__pre_init__` function which calls its superclass `__init__` function, its subclasses will inherit this `__pre_init__` function too.
- When subclasses of `AutoInit` create and initialize object intances through `__call__` from `PrePostInitMeta`, `AutoInit`'s `__pre_init__` runs `super().__init__(...)`, so when we write `__init__` function of subclasses which inherits from `AutoInit`, we don't need to write `super().__init__(...)` any more.

#### NewChkMeta

is inherited from `FixSigMeta`, so any instance classes created by `NewChkMeta` can also pass on the no_signature_problem feature.

It defines its own `__call__` to enable all the instance objects e.g., `t` created by all the instance classes e.g., `T` created by `NewChkMeta` to do the following: 

- `T(t) == t if isinstance(t, T)` returns true
- when `T(t) == t if not isinstance(t, T)`, or when `T(t, 1) == t if isinstance(t, T)` or when `T(t, b=1) == t if isinstance(t, T)`, all return False

In other words, `NewChkMeta` creates a new breed of classes which won't recreate the same instance object twice.

In [ ]:
fastview(delegates)


def low(a, b:int=1): pass
@delegates(low)
def mid(c, d:list=None, **kwargs): pass
pprint(inspect.signature(mid)) # pprint and inspect is loaded from fastdebug

def delegates(to:FunctionType=None, # Delegatee===========================================(0) # how to make delegates(to) to have to as FunctionType and default as None; 
              keep=False, # Keep `kwargs` in decorated function?==========================(1)       
              but:list=None): # Exclude these parameters from signature===================(2) # how to make delegates(to, but) to have 'but' as list and default as None; 
    "Decorator: replace `**kwargs` in signature with params from `to`"====================(3)       
    if but is None: but = []==============================================================(4)       
    def _f(f):============================================================================(5)       
        if to is None: to_f,from_f = f.__base__.__init__,f.__init__=======================(6)

In [ ]:
fastlist()

test_sig.py
BypassNewMeta.py
snoop.py
FixSigMeta.py
funcs_kwargs.py
NewChkMeta.py
printtitle.py
AutoInit.py
method.py
_rm_self.py
delegates.py
create_explore_str.py
PrePostInitMeta.py
_funcs_kwargs.py
whatinside.py


In [ ]:
fastview(FixSigMeta)


class BaseMeta(FixSigMeta): 
    # using __new__ of  FixSigMeta instead of type
    def __call__(cls, *args, **kwargs): pass

class Foo_call_fix(metaclass=BaseMeta): # Base
    def __init__(self, d, e, f): pass

pprint(inspect._signature_from_callable(Foo_call_fix, sigcls=inspect.Signature))    

class FixSigMeta(type):===================================================================(0)       
    "A metaclass that fixes the signature on classes that override `__new__`"=============(1) # Any class having FixSigMeta as metaclass will have its own __init__ func stored in its attr __signature__;FixSigMeta uses its __new__ to create a class instance; then check whether its class instance has its own __init__;if so, remove self from the sig of __init__; then assign this new sig to __signature__ for the class instance;; 
    def __new__(cls, name, bases, dict):==================================================(2) # how does a metaclass create a class instance; what does super().__new__() 

In [ ]:
fastsearch("how to get signature's parameters", nb=True)

Found a line: ========>
        s2 = {k:v.replace(kind=inspect.Parameter.KEYWORD_ONLY) for k,v in inspect.signature(to_f).parameters.items() # How to access a signature's parameters as a dict?; How to replace the kind of a parameter with a different kind?; how to check whether a parameter has a default value?; How to check whether a string is in a dict and a list?; how dict.items() and dict.values() differ;  (14)

The source where the line is from ==========>

def low(a, b:int=1): pass
@delegates(low)
def mid(c, d:list=None, **kwargs): pass
pprint(inspect.signature(mid)) # pprint and inspect is loaded from fastdebug

def delegates(to:FunctionType=None, # Delegatee===========================================(0) # how to make delegates(to) to have to as FunctionType and default as None; 
              keep=False, # Keep `kwargs` in decorated function?==========================(1)       
              but:list=None): # Exclude these parameters from signature===================(2) # how to 

[Open `delegates` in Jupyter Notebook](http://localhost:8888/tree/nbs/demos/0001_fastcore_meta_delegates.ipynb)

## What is fastcore.meta all about